Kaggle link: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_KEY")

import wandb
wandb.login(key=secret_value_0)
wandb.init(project='DogsVsCatsCNN', save_code=True)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import everything needed

In [ ]:
import zipfile
import glob
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Unzip datasets

In [ ]:
train_dir = 'train'
test_dir = 'test'
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip') as train_zip:
    train_zip.extractall('')
    
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip') as test_zip:
    test_zip.extractall('')
train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

In [ ]:
labels = [path.split('/')[-1].split('.')[0] for path in train_list]

## Plot random image with their label

In [ ]:
random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx])
    ax.set_title(labels[idx])
    ax.imshow(img)

## Use Sklearn to split data

In [ ]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=labels,
                                          random_state=0)
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

We will discuss this in more detail in a near future...

In [ ]:
train_transforms = transforms.Compose([
        transforms.Resize(128), # makes it easier for the GPU
        transforms.RandomResizedCrop(112),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

val_transforms = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(112),
        transforms.ToTensor()])


test_transforms = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(112),
        transforms.ToTensor()])

Define the dataset using PIL to read image

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        self.filelength = len(file_list)

    def __len__(self):
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0
        return img_transformed, label

In [ ]:
train_data = CatsDogsDataset(train_list, transform=train_transforms)
valid_data = CatsDogsDataset(valid_list, transform=test_transforms)
test_data = CatsDogsDataset(test_list, transform=test_transforms)

Create dataloader, you can modify the batch size if needed

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight, nn.init.calculate_gain('relu'))
        torch.nn.init.constant_(m.bias, 0)


# AlexNet

In 2012, Alex Krizhevsky, Ilya Sutskever and Geoffrey Hinton proposed a CNN architecture and implemented it on Cuda GPUs to compete in the ImageNet competition.

In [ ]:

class AlexNet(nn.Module):
    
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1  = nn.Linear(in_features= 1024, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 4096)
        self.fc3 = nn.Linear(in_features=4096 , out_features=1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool3(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.sigmoid(x)

In [ ]:


model = AlexNet()
model.apply(init_weights)

num_epochs = 100
lr =  0.01
weight_decay = 1e-4

optimizer = torch.optim.SGD(model.parameters(),lr = lr)#torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay) #AdamW to aviod coupling problem between the L2 regularization and the adaptive learning rate
criterion = torch.nn.BCELoss()

#logging configuration and model
wandb.config.learningrate = lr
wandb.config.num_iterations = num_epochs
wandb.config.optimizer = "SGD"
wandb.config.criterion = "BCELoss"

wandb.watch(model, log="all", criterion=criterion, log_freq=1,  log_graph=(True)) #log frequency depend on your training


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #0, first GPU if multiple one
print(device)
model.to(device)

# Training Loop

In [ ]:
for epoch in range(num_epochs):
    
    #TRAINING LOOP
    training_loss = 0
    correct_classified = 0
    model.train()
    for train_features, train_labels in train_loader:
        optimizer.zero_grad() # please don't forget!
        
        out = model(train_features)
        l = criterion(out, train_labels.unsqueeze(1).type(torch.FloatTensor))
        
        l.backward() # compute gradient
        optimizer.step() # do a step in the gradient direction
        
        training_loss += l.item()
        #Sum correct classified...
        _, predicted = torch.max(out.data, 1)
        correct_classified += int(predicted.eq(train_labels.unsqueeze(1)).sum().item())
        
    train_acc = correct_classified / training_size
    
    #VALIDATION LOOP
    validation_loss = 0
    correct_classified = 0
    model.eval()
    with torch.no_grad():
        for val_features, val_labels in valid_loader:
            out = model(val_features)
            l = criterion(out, val_labels.unsqueeze(1))
            
            validation_loss += l.item()
            #Sum correct classified...
            predicted = torch.ge(out, 0.5)
            correct_classified += int(predicted.eq(val_labels.unsqueeze(1)).sum().item())
            
    val_acc = correct_classified / validation_size         
    wandb.log({'training_loss': training_loss, 'validation_loss': validation_loss, 'validation_accuracy': val_acc, 'training_accuracy': train_acc})
    
    
# SAVE THE MODEL
torch.save(model.state_dict(), 'my_model_' + wandb.run.name + '_' + wandb.run.id +'.pt')
print("finished...")